In [46]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Import dataset
aurora_cara = pd.read_csv("Aurora v2.1 data file - caracterización.csv")
aurora_feedback = pd.read_csv("Aurora v2.1 data file - ayudaHumanitaria.csv")
aurora_monitoreo = pd.read_csv("Aurora v2.1 data file - monitoreo.csv")

In [47]:
aurora = pd.merge(aurora_cara, aurora_feedback)
aurora.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 356
Data columns (total 70 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ¿Quién entra al sistema?                                357 non-null    object 
 1   ¿Cómo interactúa con el sistema?                        357 non-null    object 
 2   Tipo de enganche                                        0 non-null      float64
 3   UserId                                                  357 non-null    int64  
 4   InteraID                                                357 non-null    object 
 5   Inicio interacción                                      357 non-null    object 
 6   Final interacción                                       0 non-null      float64
 7   Tiempo de interacción                                   110 non-null    float64
 8   Alguien te está ayudando a conectarte   

In [48]:
aurora_comple= aurora.append(aurora_monitoreo)

C:\Users\maola\AppData\Local\Temp\ipykernel_31476\1469997505.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aurora_comple= aurora.append(aurora_monitoreo)


In [49]:
# Drop observations of Aurora team phones and test registers 
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311571598].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311398466 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311396734 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361421 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361350 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311361257 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311337494 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311325070 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311325038 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311272934 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310820267 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310543580 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310357249 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 310191611 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 308421831 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 306028996 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311725039].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311719001 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311718121].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311699383 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311696700 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 312179120 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311965863 ].index)
aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == 311965863 ].index)

In [50]:
#Variable of date in date format
aurora_comple["Inicio interacción"] = pd.to_datetime(
aurora_comple["Inicio interacción"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')
#aurora = aurora[aurora["Inicio interacción"] < '2023-10-29']

In [51]:
aurora_comple= aurora_comple.sort_values(by=['UserId', 'Inicio interacción'])

# Reset the index for proper grouping
aurora_comple= aurora_comple.reset_index(drop=True)

# Create a new variable "Interaction_Sequence" based on the grouping
aurora_comple['Interaction_Sequence'] = aurora_comple.groupby('UserId').cumcount() + 1

In [52]:
aurora_comple['Interaction_Sequence']

0      1
1      2
2      3
3      4
4      1
      ..
562    1
563    1
564    1
565    1
566    1
Name: Interaction_Sequence, Length: 567, dtype: int64

In [53]:
excel_file = 'completa.xlsx'
aurora_comple.to_excel(excel_file, index=False)

In [23]:
unpivoted = aurora_comple.groupby(['¿Cómo interactúa con el sistema?', 'Inicio interacción', 'Tipo de monitoreo'])[
    'UserId'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='Inicio interacción',
    index=['¿Cómo interactúa con el sistema?', 'Tipo de monitoreo'],
    values='UserId')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()
pivoted

Inicio interacción,¿Cómo interactúa con el sistema?,Tipo de monitoreo,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-22,2023-10-23,2023-10-24,2023-10-25,2023-10-26,2023-10-27,2023-10-28,2023-10-29,2023-10-30,2023-10-31,2023-11-01
0,Monitoreo,Feedback,NaN,NaN,3.0,2.0,4.0,6.0,7.0,7.0,7.0,16.0,8.0,16.0,15.0,19.0,12.0
1,Monitoreo,NNA,1.0,NaN,2.0,5.0,4.0,2.0,NaN,4.0,3.0,1.0,3.0,3.0,1.0,3.0,1.0
2,Monitoreo,Percepción,1.0,1.0,3.0,NaN,6.0,1.0,1.0,2.0,4.0,4.0,8.0,3.0,3.0,2.0,NaN
3,Monitoreo,Vocación Permanencia,2.0,4.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [ ]:
columns = aurora_carto.columns.tolist()

In [ ]:
for column in columns:
  sns.countplot(x=column, data=aurora_carto, order = aurora_carto[column].value_counts().index)
  plt.xticks(rotation=70, fontsize=10)
  plt.xlabel(column, fontsize=12)
#  plt.title(f'[{column}]', fontsize=12)
  plt.show()
  plt.clf()